# Automatic Question Answering from FAQ using word embeddings

In [18]:
import pandas as pd;

#Load dataset and examine dataset, rename columns to questions and answers

df=pd.read_csv("KafkaFAQ.csv");
df.columns=["Questions","Answers"];

df

,Questions,Answers
0,"How to identify if a cluster is unhealthy, wha...",1
1,What happens to the quotas when there is a bro...,2
2,Document the process to open firewall from clo...,3
3,How to add wildcard access to a specific topic...,4
4,How is this billed?,5
5,Can I change the retention of my pipeline?,6
6,Why is creating a pipeline with topic Affinity...,7
7,Can I scale up my pipeline?,8
8,Can I delete a single topic on a cluster?,9
9,"What is the difference between a pipeline, a t...",10


In [19]:

import re
import gensim 
from gensim.parsing.preprocessing import remove_stopwords

#from nltk.stem.lancaster import LancasterStemmer
#st = LancasterStemmer()

def clean_sentence(sentence, stopwords=False):
    
    sentence = sentence.lower().strip()
    sentence = re.sub(r'[^a-z0-9\s]', '', sentence)
    #sentence = re.sub(r'\s{2,}', ' ', sentence)
    
    if stopwords:
         sentence = remove_stopwords(sentence)
    
    #sent_stemmed='';
    #for word in sentence.split():
    #    sent_stemmed+=' '+st.stem(word) 
    #sentence=sent_stemmed
    
    return sentence
                    
def get_cleaned_sentences(df,stopwords=False):    
    sents=df[["Questions"]];
    cleaned_sentences=[]

    for index,row in df.iterrows():
        #print(index,row)
        cleaned=clean_sentence(row["Questions"],stopwords);
        cleaned_sentences.append(cleaned);
    return cleaned_sentences;

cleaned_sentences=get_cleaned_sentences(df,stopwords=True)
print(cleaned_sentences);

print("\n")

cleaned_sentences_with_stopwords=get_cleaned_sentences(df,stopwords=False)
print(cleaned_sentences_with_stopwords);
    

['identify cluster unhealthy steps metrics look', 'happens quotas broker failure', 'document process open firewall cloud onprem vice versa', 'add wildcard access specific topic cluster', 'billed', 'change retention pipeline', 'creating pipeline topic affinity taking long', 'scale pipeline', 'delete single topic cluster', 'difference pipeline topic broker partition cluster', 'service support multiple topics single kafka cluster']


['how to identify if a cluster is unhealthy what are the first steps or metrics we should look for', 'what happens to the quotas when there is a broker failure', 'document the process to open firewall from cloud to onprem or vice versa', 'how to add wildcard access to a specific topic in a cluster', 'how is this billed', 'can i change the retention of my pipeline', 'why is creating a pipeline with topic affinity taking so long', 'can i scale up my pipeline', 'can i delete a single topic on a cluster', 'what is the difference between a pipeline a topic a broke

In [39]:
import numpy

sentences=cleaned_sentences_with_stopwords
#sentences=cleaned_sentences

# Split it by white space 
sentence_words = [[word for word in document.split() ]
         for document in sentences]

from gensim import corpora

dictionary = corpora.Dictionary(sentence_words)
for key, value in dictionary.items():
    print(key, ' : ', value)

import pprint
# bow_corpus = [dictionary.doc2bow(text) for text in sentence_words]
# for sent,embedding in zip(sentences,bow_corpus):
#     print(sent)
#     print(embedding)

question_orig="how is this priced"
question=clean_sentence(question_orig,stopwords=False);
# question_embedding = dictionary.doc2bow(question.split())

# print("\n\n",question,"\n",question_embedding)

0  :  a
1  :  are
2  :  cluster
3  :  first
4  :  for
5  :  how
6  :  identify
7  :  if
8  :  is
9  :  look
10  :  metrics
11  :  or
12  :  should
13  :  steps
14  :  the
15  :  to
16  :  unhealthy
17  :  we
18  :  what
19  :  broker
20  :  failure
21  :  happens
22  :  quotas
23  :  there
24  :  when
25  :  cloud
26  :  document
27  :  firewall
28  :  from
29  :  onprem
30  :  open
31  :  process
32  :  versa
33  :  vice
34  :  access
35  :  add
36  :  in
37  :  specific
38  :  topic
39  :  wildcard
40  :  billed
41  :  this
42  :  can
43  :  change
44  :  i
45  :  my
46  :  of
47  :  pipeline
48  :  retention
49  :  affinity
50  :  creating
51  :  long
52  :  so
53  :  taking
54  :  why
55  :  with
56  :  scale
57  :  up
58  :  delete
59  :  on
60  :  single
61  :  and
62  :  between
63  :  difference
64  :  partition
65  :  does
66  :  kafka
67  :  multiple
68  :  service
69  :  support
70  :  topics
71  :  your
how to identify if a cluster is unhealthy what are the first steps or m

In [41]:
import sklearn
from sklearn.metrics.pairwise import cosine_similarity;
def retrieveAndPrintFAQAnswer(question_embedding,sentence_embeddings,FAQdf,sentences):
    max_sim=-1;
    index_sim=-1;
    for index,faq_embedding in enumerate(sentence_embeddings):
        #sim=cosine_similarity(embedding.reshape(1, -1),question_embedding.reshape(1, -1))[0][0];
        sim=cosine_similarity(faq_embedding,question_embedding)[0][0];
        print(index, sim, sentences[index])
        if sim>max_sim:
            max_sim=sim;
            index_sim=index;
       
    print("\n")
    print("Question: ",question)
    print("\n");
    print("Retrieved: ",FAQdf.iloc[index_sim,0]) 
    print(FAQdf.iloc[index_sim,1])        
    
#retrieveAndPrintFAQAnswer(question_embedding,bow_corpus,df,sentences);

0 0.19611613513818404 how to identify if a cluster is unhealthy what are the first steps or metrics we should look for
1 0.19611613513818404 what happens to the quotas when there is a broker failure
2 0.994309153919809 document the process to open firewall from cloud to onprem or vice versa
3 0.19611613513818404 how to add wildcard access to a specific topic in a cluster
4 1.0000000000000002 how is this billed
5 0.9920614219374349 can i change the retention of my pipeline
6 0.19611613513818404 why is creating a pipeline with topic affinity taking so long
7 0.9849709602558829 can i scale up my pipeline
8 0.19611613513818404 can i delete a single topic on a cluster
9 0.19611613513818404 what is the difference between a pipeline a topic a broker a partition and a cluster
10 0.19611613513818404 does your service support multiple topics on a single kafka cluster


Question:  how is this priced


Retrieved:  How is this billed?
5


# Glove Embeddings : 

Glove is an alternate approach to build word embeddings using matrix factorization techinques on the word-word co-occurance matrix. 

https://nlp.stanford.edu/projects/glove/

While both the techniques are popular, glove performs better on some datasets while word2vec skipgram model performs better on some. Here, we experiment with both the word2vec and the glove models. 


In [42]:
from gensim.models import Word2Vec 
import gensim.downloader as api


glove_model=None;
try:
    glove_model = gensim.models.KeyedVectors.load("./glovemodel.mod")
    print("Loaded glove model")
except:            
    glove_model = api.load('glove-twitter-25')
    glove_model.save("./glovemodel.mod")
    print("Saved glove model")
    
glove_embedding_size=len(glove_model['computer']);


Loaded glove model


In [43]:
def getWordVec(word,model):
        samp=model['computer'];
        vec=[0]*len(samp);
        try:
                vec=model[word];
        except:
                vec=[0]*len(samp);
        return (vec)


def getPhraseEmbedding(phrase,embeddingmodel):
                       
        samp=getWordVec('computer', embeddingmodel);
        vec=numpy.array([0]*len(samp));
        den=0;
        for word in phrase.split():
            #print(word)
            den=den+1;
            vec=vec+numpy.array(getWordVec(word,embeddingmodel));
        #vec=vec/den;
        #return (vec.tolist());
        return vec.reshape(1, -1)


In [48]:
#With Glove

sent_embeddings=[];
for sent in cleaned_sentences:
    sent_embeddings.append(getPhraseEmbedding(sent,glove_model));
    
question_embedding=getPhraseEmbedding(question,glove_model);

retrieveAndPrintFAQAnswer(question_embedding,sent_embeddings,df, cleaned_sentences);


0 0.8349929770635101 how to identify if a cluster is unhealthy what are the first steps or metrics we should look for
1 0.7686536998263341 what happens to the quotas when there is a broker failure
2 0.7840371073746832 document the process to open firewall from cloud to onprem or vice versa
3 0.8228934099586328 how to add wildcard access to a specific topic in a cluster
4 0.2619826370675414 how is this billed
5 0.7020266814466231 can i change the retention of my pipeline
6 0.879914780346787 why is creating a pipeline with topic affinity taking so long
7 0.5647586598382187 can i scale up my pipeline
8 0.8215086809402338 can i delete a single topic on a cluster
9 0.7387740618133274 what is the difference between a pipeline a topic a broker a partition and a cluster
10 0.8504574561672683 does your service support multiple topics on a single kafka cluster


Question:  how is this priced


Retrieved:  Why is creating a pipeline with topic Affinity taking so long?
7
